### fc_25 전처리코드
1. 선수명 표기 통일
2. 선수 포지션 표기 통일
3. 선수 소속팀 표기 통일
4. 국가명에 '카보베르데제도' -> 영어표기변경
5. 중복 데이터 처리

In [9]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re

In [7]:
fc_df = pd.read_csv('../data/fc25_playerdata.csv', encoding='utf-8-sig')
fc_df

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,Kylian Mbappe,France,Real Madrid,ST,91,97,90,80,92,36,78
1,Rodri,Spain,Manchester City,CDM,91,66,80,86,84,87,85
2,Erling Haaland,Norway,Manchester City,ST,91,88,92,70,81,45,88
3,Jude Bellingham,England,Real Madrid,MOC,90,80,87,83,88,78,83
4,Vini Jr.,Brazil,Real Madrid,LW,90,95,84,81,91,29,69
...,...,...,...,...,...,...,...,...,...,...,...
15900,Engson Singh,India,Mohun Bagan SG,MOC,47,31,45,47,52,34,45
15901,Ousmane Sylla,Senegal,Houston Dynamo,CM,47,67,38,46,45,40,55
15902,Wei Feng,China,Changchun Yatai,RB,47,57,47,38,50,43,49
15903,Won Hui Do,"Korea, South",Jeju United,CM,47,66,35,46,46,41,64


In [5]:
player_df = pd.read_csv('../data/tfm_finaldata.csv', encoding='utf-8-sig')
player_df

,name,field,position,age,nation,confederation,club,league,value_billion_krw
0,Lamine Yamal,FW,Right_Winger,18,Spain,UEFA,FC Barcelona,LaLiga,3250.0
1,Erling Haaland,FW,Striker,25,Norway,UEFA,Manchester City,PremierLeague,2925.0
2,Kylian Mbappe,FW,Striker,26,France,UEFA,Real Madrid,LaLiga,2925.0
3,Jude Bellingham,MF,Attacking_Midfielder,22,England,UEFA,Real Madrid,LaLiga,2925.0
4,Vinicius Junior,FW,Left_Winger,25,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5
...,...,...,...,...,...,...,...,...,...
5348,Vuniuci Tikomaimereke,DF,Left_Back,35,Fiji,OFC,Nadroga FC,Fiji_PremierLeague,0.8
5349,Frank Clarke,FW,Striker,27,New Zealand,OFC,Fencibles United AFC,NewZealand_FootballLeague,0.8
5350,Emori Ragata,GK,Goalkeeper,37,Fiji,OFC,Nasinu FC,Fiji_PremierLeague,0.8
5351,Kazik Swain,MF,Attacking_Midfielder,19,New Zealand,OFC,Bay Olympic FC,NewZealand_FootballLeague,0.8


- 선수 이름 표기 통일

In [8]:
fc_kor = fc_df[fc_df['nation'] == 'Korea, South']
fc_kor.head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
38,Heung Min Son,"Korea, South",Spurs,ST,87,87,89,81,85,42,73
151,Kim Min Jae,"Korea, South",FC Bayern München,CB,83,76,33,57,63,85,83
542,Lee Kang In,"Korea, South",Paris SG,MOC,79,74,74,80,82,50,66
706,Hwang Hee Chan,"Korea, South",Wolves,ST,78,84,77,70,78,39,74
1187,Lee Jae Sung,"Korea, South",1. FSV Mainz 05,MOC,76,69,71,72,81,59,61


In [6]:
tfm_kor = player_df[player_df['nation'] == 'Korea, South']
tfm_kor.head() 
# transfermarkt 데이터의 한국인 선수 표기명이 다른걸 확인

,name,field,position,age,nation,confederation,club,league,value_billion_krw
163,Min-jae Kim,DF,Centre_Back,28,"Korea, South",AFC,Bayern Munich,Bundesliga,650.0
351,Kang-in Lee,FW,Right_Winger,24,"Korea, South",AFC,Paris Saint-Germain,Ligue1,406.2
521,Son Heung Min,FW,Left_Winger,33,"Korea, South",AFC,Tottenham Hotspur,PremierLeague,325.0
1011,Hee-chan Hwang,FW,Striker,29,"Korea, South",AFC,Wolverhampton Wanderers,PremierLeague,195.0
1157,In-beom Hwang,MF,Central_Midfielder,28,"Korea, South",AFC,Feyenoord Rotterdam,Eredivisie,162.5


- 한국인 선수들 이름 표기를 FC25 방식으로 통일 (ex. Min-jae Kim -> Kim Min Jae)

In [10]:
# 이름 변환 함수
def convert_name(name):
    parts = name.strip().split()
    if len(parts) != 2: # 이름이 2단어가 아니면 변환하지 않음
        return name
    first_name = parts[0].replace('-', ' ').title()
    last_name = parts[1].title()
    return f"{last_name} {first_name}"

# 한국 국적 선수에게만 이름 변환 적용
player_df.loc[player_df['nation'] == 'Korea, South', 'name'] = (player_df.loc[player_df['nation'] == 'Korea, South', 'name'].apply(convert_name))

In [ ]:
# 적용 확인용
player_df[player_df['nation'] == 'Korea, South'].head()

,name,field,position,age,nation,confederation,club,league,value_billion_krw
163,Kim Min Jae,DF,Centre_Back,28,"Korea, South",AFC,Bayern Munich,Bundesliga,650.0
351,Lee Kang In,FW,Right_Winger,24,"Korea, South",AFC,Paris Saint-Germain,Ligue1,406.2
521,Son Heung Min,FW,Left_Winger,33,"Korea, South",AFC,Tottenham Hotspur,PremierLeague,325.0
1011,Hwang Hee Chan,FW,Striker,29,"Korea, South",AFC,Wolverhampton Wanderers,PremierLeague,195.0
1157,Hwang In Beom,MF,Central_Midfielder,28,"Korea, South",AFC,Feyenoord Rotterdam,Eredivisie,162.5


In [179]:
# 이름 정규화 함수
def normalize_name(name):
    name = name.lower().replace('-', ' ').replace('.', '').strip()
    name_parts = name.split()
    return ''.join(sorted(name_parts))  # 알파벳 순 정렬하여 비교

# 정규화한 이름 컬럼 추가
fc_df['name_normalized'] = fc_df['name'].apply(normalize_name)
player_df['name_normalized'] = player_df['name'].apply(normalize_name)

# 공통된 정규화 이름 비교
common_names = set(fc_df['name_normalized']) & set(player_df['name_normalized'])

In [180]:
matched = fc_df[fc_df['name_normalized'].isin(common_names)][['name']]
print(matched)

                  name
0        Kylian Mbappe
1                Rodri
2       Erling Haaland
3      Jude Bellingham
4      Vinicius Junior
...                ...
15679      Tyron Owusu
15762  Jovan Mijatovic
15819      Kim Jae Min
15878           Wu Lei
15889     Isaiah Jones

[3465 rows x 1 columns]


In [181]:
player_df.to_csv('../data/nor_tfm.csv', index=False, encoding='utf-8-sig')
player_df

,name,field,position,age,nation,confederation,club,league,value_billion_krw,name_normalized
0,Lamine Yamal,FW,Right_Winger,18,Spain,UEFA,FC Barcelona,LaLiga,3250.0,lamineyamal
1,Erling Haaland,FW,Striker,25,Norway,UEFA,Manchester City,PremierLeague,2925.0,erlinghaaland
2,Kylian Mbappe,FW,Striker,26,France,UEFA,Real Madrid,LaLiga,2925.0,kylianmbappe
3,Jude Bellingham,MF,Attacking_Midfielder,22,England,UEFA,Real Madrid,LaLiga,2925.0,bellinghamjude
4,Vinicius Junior,FW,Left_Winger,25,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5,juniorvinicius
...,...,...,...,...,...,...,...,...,...,...
5348,Vuniuci Tikomaimereke,DF,Left_Back,35,Fiji,OFC,Nadroga FC,Fiji_PremierLeague,0.8,tikomaimerekevuniuci
5349,Frank Clarke,FW,Striker,27,New Zealand,OFC,Fencibles United AFC,NewZealand_FootballLeague,0.8,clarkefrank
5350,Emori Ragata,GK,Goalkeeper,37,Fiji,OFC,Nasinu FC,Fiji_PremierLeague,0.8,emoriragata
5351,Kazik Swain,MF,Attacking_Midfielder,19,New Zealand,OFC,Bay Olympic FC,NewZealand_FootballLeague,0.8,kazikswain


In [182]:
fc_df.to_csv('../data/nor_fc25.csv', index=False, encoding='utf-8-sig')
fc_df

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,name_normalized
0,Kylian Mbappe,France,Real Madrid,Striker,91,97,90,80,92,36,78,kylianmbappe
1,Rodri,Spain,Manchester City,Defensive_Midfielder,91,66,80,86,84,87,85,rodri
2,Erling Haaland,Norway,Manchester City,Striker,91,88,92,70,81,45,88,erlinghaaland
3,Jude Bellingham,England,Real Madrid,Attacking_Midfielder,90,80,87,83,88,78,83,bellinghamjude
4,Vinicius Junior,Brazil,Real Madrid,Left_Winger,90,95,84,81,91,29,69,juniorvinicius
...,...,...,...,...,...,...,...,...,...,...,...,...
15900,Engson Singh,India,Mohun Bagan SG,Attacking_Midfielder,47,31,45,47,52,34,45,engsonsingh
15901,Ousmane Sylla,Senegal,Houston Dynamo,Central_Midfielder,47,67,38,46,45,40,55,ousmanesylla
15902,Wei Feng,China,Changchun Yatai,Right_Back,47,57,47,38,50,43,49,fengwei
15903,Won Hui Do,"Korea, South",Jeju United,Central_Midfielder,47,66,35,46,46,41,64,dohuiwon


- transfermarkt데이터와 fc25데이터의 선수 표기명이 일부 다른걸 확인
- ex) 'Min-Jae Kim' 과 'Kim Min Jae'
- 표기가 다른 이름을 전부 정규화를 시켜서 'name_normalized'라는 칼럼을 새로 만듦 -> 나중에 국적이나 포지션을 엮어서 id화 할수 있을듯

In [183]:
fc_df[fc_df.duplicated(subset=['name','nation', 'position'], keep=False)].sort_values('name')

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,name_normalized
942,Antony,Brazil,Man Utd,Side_Midfielder,77,82,70,72,82,43,72,antony
4768,Antony,Brazil,Portland Timbers,Side_Midfielder,69,88,58,61,70,40,55,antony
414,David Lopez,Spain,Girona FC,Centre_Back,80,45,60,71,66,82,75,davidlopez
9213,David Lopez,Spain,Burgos CF,Centre_Back,65,65,31,46,51,63,78,davidlopez
5953,Diego Gonzalez,Spain,Albacete BP,Centre_Back,68,65,34,60,57,67,74,diegogonzalez
10036,Diego Gonzalez,Spain,SD Huesca,Centre_Back,64,61,42,54,56,65,63,diegogonzalez
6774,Jesus Castillo,Peru,Gil Vicente,Defensive_Midfielder,67,56,57,64,63,61,69,castillojesus
6775,Jesus Castillo,Peru,Alianza Lima,Defensive_Midfielder,67,61,58,59,61,65,66,castillojesus
913,Joao Pedro,Brazil,Brighton,Striker,77,77,77,67,79,28,62,joaopedro
2495,Joao Pedro,Brazil,Al Taawoun,Striker,73,65,75,53,67,25,80,joaopedro


In [184]:
player_df[player_df.duplicated(subset=['name','nation', 'position'], keep=False)].sort_values('name')
# 'name','nation', 'position' 의 조합으로 중복을 찾았는데도 동일 선수가 2명이 나옴, 추가 필터링이 필요함

,name,field,position,age,nation,confederation,club,league,value_billion_krw,name_normalized
767,Fabinho,MF,Defensive_Midfielder,31,Brazil,CONMEBOL,Al-Ittihad Club,Saudi_ProLeague,243.8,fabinho
1976,Fabinho,MF,Defensive_Midfielder,23,Brazil,CONMEBOL,Red Bull Bragantino,Brasileiro_SerieA,81.2,fabinho
1218,Wesley,FW,Left_Winger,26,Brazil,CONMEBOL,Sport Club Internacional,Brasileiro_SerieA,146.2,wesley
2363,Wesley,FW,Left_Winger,20,Brazil,CONMEBOL,Al-Nassr FC,Saudi_ProLeague,65.0,wesley


In [185]:
player_df[player_df.duplicated(subset=['name','nation', 'position','club'], keep=False)].sort_values('name')


,name,field,position,age,nation,confederation,club,league,value_billion_krw,name_normalized


In [186]:
fc_df[fc_df.duplicated(subset=['name','nation', 'position','club'], keep=False)].sort_values('name')

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,name_normalized


In [187]:
# 포지션 표기명 통일화

position_mapping = {
    # Striker
    'Centre-Forward': 'Striker', 'Second Striker': 'Striker', 'ST': 'Striker', 'CF': 'Striker',

    # Winger
    'Right Winger': 'Right_Winger', 'RW': 'Right_Winger',
    'Left Winger': 'Left_Winger', 'LW': 'Left_Winger',
    'Attack': '(Uncategorized)Forwards',

    # Midfield
    'Attacking Midfield': 'Attacking_Midfielder', 'MOC': 'Attacking_Midfielder',
    'Central Midfield': 'Central_Midfielder', 'CM': 'Central_Midfielder',
    'Right Midfield': 'Side_Midfielder', 'Left Midfield': 'Side_Midfielder',
    'LM': 'Side_Midfielder', 'RM': 'Side_Midfielder',
    'Defensive Midfield': 'Defensive_Midfielder', 'CDM': 'Defensive_Midfielder',
    'Midfield': '(Uncategorized)Midfield',

    # Defender
    'Centre-Back': 'Centre_Back', 'Sweeper': 'Centre_Back', 'CB': 'Centre_Back',
    'Right-Back': 'Right_Back', 'RB': 'Right_Back',
    'Left-Back': 'Left_Back', 'LB': 'Left_Back',
    'Defender': '(Uncategorized)Defender',

    # Goalkeeper
    'Goalkeeper': 'Goalkeeper', 'GK': 'Goalkeeper'
}

fc_df['position'] = fc_df['position'].replace(position_mapping)
fc_df

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,name_normalized
0,Kylian Mbappe,France,Real Madrid,Striker,91,97,90,80,92,36,78,kylianmbappe
1,Rodri,Spain,Manchester City,Defensive_Midfielder,91,66,80,86,84,87,85,rodri
2,Erling Haaland,Norway,Manchester City,Striker,91,88,92,70,81,45,88,erlinghaaland
3,Jude Bellingham,England,Real Madrid,Attacking_Midfielder,90,80,87,83,88,78,83,bellinghamjude
4,Vinicius Junior,Brazil,Real Madrid,Left_Winger,90,95,84,81,91,29,69,juniorvinicius
...,...,...,...,...,...,...,...,...,...,...,...,...
15900,Engson Singh,India,Mohun Bagan SG,Attacking_Midfielder,47,31,45,47,52,34,45,engsonsingh
15901,Ousmane Sylla,Senegal,Houston Dynamo,Central_Midfielder,47,67,38,46,45,40,55,ousmanesylla
15902,Wei Feng,China,Changchun Yatai,Right_Back,47,57,47,38,50,43,49,fengwei
15903,Won Hui Do,"Korea, South",Jeju United,Central_Midfielder,47,66,35,46,46,41,64,dohuiwon


In [188]:
# fc25 데이터에 '카보베르데제도' 국가명 transfermarkt기준에 맞게 재할당
fc_df['nation'] = fc_df['nation'].replace('카보베르데제도', 'Cabo Verde')
fc_df[fc_df['nation'] == 'Cabo Verde'].head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,name_normalized
1613,Bruno Varela,Cabo Verde,Vitória SC,Goalkeeper,75,75,73,69,76,39,73,brunovarela
1764,Logan Costa,Cabo Verde,Villarreal CF,Centre_Back,74,48,38,55,61,75,75,costalogan
2260,Bebe,Cabo Verde,Racing de Ferrol,Side_Midfielder,73,81,76,70,71,41,83,bebe
2805,Nuno da Costa,Cabo Verde,Kasımpaşa,Striker,72,82,69,63,72,27,65,costadanuno
4379,Leandro Andrade,Cabo Verde,Qarabağ FK,Right_Winger,70,75,68,63,73,29,60,andradeleandro


In [189]:
# 추가로 전체 포지션 분류
FW = ['Striker', 'Right_Winger', 'Left_Winger', '(Uncategorized)Forwards']
MF = ['Attacking_Midfielder', 'Side_Midfielder', 'Central_Midfielder', 'Defensive_Midfielder', '(Uncategorized)Midfield']
DF = ['Centre_Back', 'Right_Back', 'Left_Back', '(Uncategorized)Defender']
GK = ['Goalkeeper']

conditions = [
    fc_df['position'].isin(FW),
    fc_df['position'].isin(MF),
    fc_df['position'].isin(DF),
    fc_df['position'].isin(GK)
]
choices = ['FW', 'MF', 'DF', 'GK']

fc_df['field'] = np.select(conditions, choices, default='Unknown')
fc_df.head()

,name,nation,club,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY,name_normalized,field
0,Kylian Mbappe,France,Real Madrid,Striker,91,97,90,80,92,36,78,kylianmbappe,FW
1,Rodri,Spain,Manchester City,Defensive_Midfielder,91,66,80,86,84,87,85,rodri,MF
2,Erling Haaland,Norway,Manchester City,Striker,91,88,92,70,81,45,88,erlinghaaland,FW
3,Jude Bellingham,England,Real Madrid,Attacking_Midfielder,90,80,87,83,88,78,83,bellinghamjude,MF
4,Vinicius Junior,Brazil,Real Madrid,Left_Winger,90,95,84,81,91,29,69,juniorvinicius,FW


In [190]:
new_column_order = ['name_normalized','name','nation','club','field','position','OVR','PAC','SHO','PAS','DRI','DEF','PHY']
fc_df = fc_df[new_column_order]
fc_df.head()

,name_normalized,name,nation,club,field,position,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,kylianmbappe,Kylian Mbappe,France,Real Madrid,FW,Striker,91,97,90,80,92,36,78
1,rodri,Rodri,Spain,Manchester City,MF,Defensive_Midfielder,91,66,80,86,84,87,85
2,erlinghaaland,Erling Haaland,Norway,Manchester City,FW,Striker,91,88,92,70,81,45,88
3,bellinghamjude,Jude Bellingham,England,Real Madrid,MF,Attacking_Midfielder,90,80,87,83,88,78,83
4,juniorvinicius,Vinicius Junior,Brazil,Real Madrid,FW,Left_Winger,90,95,84,81,91,29,69


In [194]:
fc_df.to_csv('../data/fc25_game_data.csv', index=False, encoding='utf-8-sig') # fc25 최종 정규화

In [192]:
new_column_order = ['name_normalized','name','age','field','position','nation','confederation','club','league','value_billion_krw']
player_df = player_df[new_column_order]
player_df.head()

,name_normalized,name,age,field,position,nation,confederation,club,league,value_billion_krw
0,lamineyamal,Lamine Yamal,18,FW,Right_Winger,Spain,UEFA,FC Barcelona,LaLiga,3250.0
1,erlinghaaland,Erling Haaland,25,FW,Striker,Norway,UEFA,Manchester City,PremierLeague,2925.0
2,kylianmbappe,Kylian Mbappe,26,FW,Striker,France,UEFA,Real Madrid,LaLiga,2925.0
3,bellinghamjude,Jude Bellingham,22,MF,Attacking_Midfielder,England,UEFA,Real Madrid,LaLiga,2925.0
4,juniorvinicius,Vinicius Junior,25,FW,Left_Winger,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5


In [195]:
player_df.to_csv('../data/tfm_player_data.csv', index=False, encoding='utf-8-sig') # transfermarkt 최종 정규화

#### 중복 데이터 필터링

In [97]:
# name, position, club도 같이 추출하여 병합
matched_fc = fc_df[fc_df['name_normalized'].isin(common_names)][['name', 'field', 'position', 'nation', 'club', 'name_normalized']]
matched_tfm = player_df[player_df['name_normalized'].isin(common_names)][['name', 'field', 'position', 'nation', 'club', 'name_normalized']]

matched_pairs = pd.merge(matched_fc, matched_tfm, on='name_normalized', suffixes=('_fc25', '_tfm'))

matched_pairs.to_csv('../data/matched_pairs.csv', index=False, encoding='utf-8-sig')
matched_pairs

,name_fc25,field_fc25,position_fc25,nation_fc25,club_fc25,name_normalized,name_tfm,field_tfm,position_tfm,nation_tfm,club_tfm
0,Kylian Mbappe,FW,Striker,France,Real Madrid,kylianmbappe,Kylian Mbappe,FW,Striker,France,Real Madrid
1,Rodri,MF,Defensive_Midfielder,Spain,Manchester City,rodri,Rodri,MF,Defensive_Midfielder,Spain,Manchester City
2,Erling Haaland,FW,Striker,Norway,Manchester City,erlinghaaland,Erling Haaland,FW,Striker,Norway,Manchester City
3,Jude Bellingham,MF,Attacking_Midfielder,England,Real Madrid,bellinghamjude,Jude Bellingham,MF,Attacking_Midfielder,England,Real Madrid
4,Vinicius Junior,FW,Left_Winger,Brazil,Real Madrid,juniorvinicius,Vinicius Junior,FW,Left_Winger,Brazil,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...
3514,Tyron Owusu,MF,Central_Midfielder,Switzerland,FC Luzern,owusutyron,Tyron Owusu,MF,Central_Midfielder,Switzerland,FC Luzern
3515,Jovan Mijatovic,FW,Striker,Serbia,New York City FC,jovanmijatovic,Jovan Mijatovic,FW,Striker,Serbia,Oud-Heverlee Leuven
3516,Kim Jae Min,MF,Central_Midfielder,"Korea, South",Jeju United,jaekimmin,Min-jae Kim,DF,Centre_Back,"Korea, South",Bayern Munich
3517,Wu Lei,MF,Defensive_Midfielder,China,Rongcheng FC,leiwu,Lei Wu,FW,Right_Winger,China,Shanghai Port


In [123]:
# 필터링 전
print(f"transfermarkt 데이터: {len(player_df)}")
print(f"FC25 데이터: {len(fc_df)}")

transfermarkt 데이터: 5353
FC25 데이터: 15905


In [124]:
common_names = set(player_df['name_normalized']) & set(fc_df['name_normalized'])

# 공통 name_normalized 값만 필터링
player_df = player_df[player_df['name_normalized'].isin(common_names)].reset_index(drop=True)
fc_df = fc_df[fc_df['name_normalized'].isin(common_names)].reset_index(drop=True)

# 필터링 후
print(f"transfermarkt 데이터: {len(player_df)}")
print(f"FC25 데이터: {len(fc_df)}")

transfermarkt 데이터: 3437
FC25 데이터: 3465


In [ ]:
# 1차 정제완료
player_df.to_csv('../data/cleaning_tfm1.csv', index=False, encoding='utf-8-sig')
fc_df.to_csv('../data/cleaning_fc1.csv', index=False, encoding='utf-8-sig')

In [131]:
# tfm 데이터에 unicode처리되지 못한 국가명 'Türkiye'발견 -> Turkey로 재할당
player_df['nation'] = player_df['nation'].replace('Türkiye', 'Turkey')
player_df[player_df['nation'] == 'Turkey'].head()

,name_normalized,name,age,field,position,nation,confederation,club,league,value_billion_krw
93,kenanyildiz,Kenan Yildiz,20,FW,Left_Winger,Turkey,UEFA,Juventus FC,Serie_A,812.5
110,ardaguler,Arda Guler,20,FW,Right_Winger,Turkey,UEFA,Real Madrid,LaLiga,731.2
244,calhanogluhakan,Hakan Calhanoglu,31,MF,Defensive_Midfielder,Turkey,UEFA,Inter Milan,Serie_A,487.5
264,kokcuorkun,Orkun Kokcu,24,MF,Central_Midfielder,Turkey,UEFA,Besiktas JK,SuperLig,487.5
369,ferdikadioglu,Ferdi Kadioglu,25,DF,Left_Back,Turkey,UEFA,Brighton & Hove Albion,PremierLeague,406.2


In [132]:
merged_df = pd.merge(fc_df, player_df, on='name_normalized', suffixes=('_fc25', '_tfm'))

# 이름은 같지만 nation이 일치하지 않는 경우 추출
mismatch_nation = merged_df[merged_df['nation_fc25'] != merged_df['nation_tfm']]
mismatch_nation

,name_normalized,name_fc25,nation_fc25,club_fc25,field_fc25,position_fc25,OVR,PAC,SHO,PAS,...,PHY,name_tfm,age,field_tfm,position_tfm,nation_tfm,confederation,club_tfm,league,value_billion_krw
46,emilianomartinez,Emiliano Martinez,Argentina,Aston Villa,GK,Goalkeeper,87,84,82,83,...,87,Emiliano Martinez,25,MF,Defensive_Midfielder,Uruguay,CONMEBOL,Sociedade Esportiva Palmeiras,Brasileiro_SerieA,113.8
202,otavio,Otavio,Portugal,Al Nassr,MF,Attacking_Midfielder,82,78,71,81,...,74,Otavio,23,DF,Centre_Back,Brazil,CONMEBOL,Paris FC,Ligue1,162.5
251,luissuarez,Luis Suarez,Uruguay,Inter Miami CF,FW,Striker,82,64,85,78,...,75,Luis Suarez,27,FW,Striker,Colombia,CONMEBOL,UD Almería,LaLiga_2,130.0
398,andresilva,Andre Silva,Portugal,RB Leipzig,FW,Striker,80,74,80,71,...,77,Andre Silva,28,FW,Striker,Brazil,CONMEBOL,São Paulo Futebol Clube,Brasileiro_SerieA,81.2
409,auderoemil,Emil Audero,Italy,Como,GK,Goalkeeper,79,80,74,73,...,76,Emil Audero,28,GK,Goalkeeper,Indonesia,AFC,Como 1907,Serie_A,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3460,oluwaseyitani,Tani Oluwaseyi,Nigeria,Minnesota United,FW,Striker,59,76,61,43,...,60,Tani Oluwaseyi,25,FW,Striker,Canada,CONCACAF,Minnesota United FC,MLS,81.2
3500,aminchiakha,Amin Chiakha,Denmark,F.C. København,FW,Striker,54,61,55,45,...,60,Amin Chiakha,19,FW,Striker,Algeria,CAF,FC Copenhagen,Danish_Superligaen,48.8
3507,pecksydie,Sydie Peck,Wales,Sheffield Utd,MF,Central_Midfielder,53,69,46,53,...,54,Sydie Peck,20,MF,Central_Midfielder,England,UEFA,Sheffield United,Championship,121.9
3512,erikkojzek,Erik Kojzek,Slovenia,Wolfsberger AC,FW,Striker,52,63,54,39,...,51,Erik Kojzek,19,FW,Striker,Austria,UEFA,Wolfsberger AC,Austria_Bundesliga,48.8


In [134]:
# 제거 대상 name_normalized만 추출
mismatch_names = mismatch_nation['name_normalized']

# 해당 name_normalized를 기준으로 제거
player_df = player_df[~player_df['name_normalized'].isin(mismatch_names)].reset_index(drop=True)
fc_df = fc_df[~fc_df['name_normalized'].isin(mismatch_names)].reset_index(drop=True)

# 필터링 후
print(f"필터링 후 transfermarkt 데이터: {len(player_df)}")
print(f"필터링 후 FC25 데이터: {len(fc_df)}")

필터링 후 transfermarkt 데이터: 3301
필터링 후 FC25 데이터: 3309


In [135]:
# 2차 정제완료
player_df.to_csv('../data/cleaning_tfm2.csv', index=False, encoding='utf-8-sig')
fc_df.to_csv('../data/cleaning_fc2.csv', index=False, encoding='utf-8-sig')

In [147]:
duplicate_tfm = fc_df[fc_df.duplicated(subset=['name', 'nation'], keep=False)]

if not duplicate_tfm.empty:
    print("name과 nation이 모두 동일한 중복 데이터 목록:")
    print(duplicate_tfm.sort_values(by=['name', 'nation']))

name과 nation이 모두 동일한 중복 데이터 목록:
     name_normalized             name        nation              club field  \
742           antony           Antony        Brazil           Man Utd    MF   
2261          antony           Antony        Brazil  Portland Timbers    MF   
768           arthur           Arthur        Brazil          Juventus    MF   
2310          arthur           Arthur        Brazil        Leverkusen    DF   
733        chiquinho        Chiquinho      Portugal     Olympiacos FC    MF   
1865       chiquinho        Chiquinho      Portugal      RCD Mallorca    FW   
190           danilo           Danilo        Brazil          Juventus    DF   
953           danilo           Danilo        Brazil     Nott'm Forest    MF   
1353          danilo           Danilo        Brazil           Rangers    FW   
2046          danilo           Danilo        Brazil    1. FC Nürnberg    DF   
355       davidlopez      David Lopez         Spain         Girona FC    DF   
2914      davidlopez

In [146]:
duplicate_fc = fc_df[fc_df.duplicated(subset=['name', 'nation'], keep=False)]

if not duplicate_fc.empty:
    print("name과 nation이 모두 동일한 중복 데이터 목록:")
    print(duplicate_fc.sort_values(by=['name', 'nation']))

name과 nation이 모두 동일한 중복 데이터 목록:
     name_normalized             name        nation              club field  \
742           antony           Antony        Brazil           Man Utd    MF   
2261          antony           Antony        Brazil  Portland Timbers    MF   
768           arthur           Arthur        Brazil          Juventus    MF   
2310          arthur           Arthur        Brazil        Leverkusen    DF   
733        chiquinho        Chiquinho      Portugal     Olympiacos FC    MF   
1865       chiquinho        Chiquinho      Portugal      RCD Mallorca    FW   
190           danilo           Danilo        Brazil          Juventus    DF   
953           danilo           Danilo        Brazil     Nott'm Forest    MF   
1353          danilo           Danilo        Brazil           Rangers    FW   
2046          danilo           Danilo        Brazil    1. FC Nürnberg    DF   
355       davidlopez      David Lopez         Spain         Girona FC    DF   
2914      davidlopez

In [148]:
tfm_df_unique = player_df.drop_duplicates(subset=['name', 'nation'], keep='first')
tfm_df_unique.to_csv('../data/cleaning_tfm.csv', index=False, encoding='utf-8-sig')

print(f"삭제 전 데이터 수: {len(player_df)}")
print(f"삭제 후 데이터 수: {len(tfm_df_unique)}")

삭제 전 데이터 수: 3301
삭제 후 데이터 수: 3282


In [149]:
fc_df_unique = fc_df.drop_duplicates(subset=['name', 'nation'], keep='first')
fc_df_unique.to_csv('../data/cleaning_fc.csv', index=False, encoding='utf-8-sig')

print(f"삭제 전 데이터 수: {len(fc_df)}")
print(f"삭제 후 데이터 수: {len(fc_df_unique)}")

삭제 전 데이터 수: 3309
삭제 후 데이터 수: 3282


In [150]:
transfer_df = pd.read_csv('../data/cleaning_tfm.csv')
fc25_df = pd.read_csv('../data/cleaning_fc.csv')

In [151]:
transfer_df.head()

,name_normalized,name,age,field,position,nation,confederation,club,league,value_billion_krw
0,lamineyamal,Lamine Yamal,18,FW,Right_Winger,Spain,UEFA,FC Barcelona,LaLiga,3250.0
1,erlinghaaland,Erling Haaland,25,FW,Striker,Norway,UEFA,Manchester City,PremierLeague,2925.0
2,kylianmbappe,Kylian Mbappe,26,FW,Striker,France,UEFA,Real Madrid,LaLiga,2925.0
3,bellinghamjude,Jude Bellingham,22,MF,Attacking_Midfielder,England,UEFA,Real Madrid,LaLiga,2925.0
4,juniorvinicius,Vinicius Junior,25,FW,Left_Winger,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5


In [152]:
# 두 데이터프레임 병합
merged_transfer_fc_df = pd.merge(transfer_df, fc_df, on='name_normalized', suffixes=('_transfer', '_fc25'))
print(f"병합된 선수 수: {len(merged_transfer_fc_df)}")

병합된 선수 수: 3310


In [159]:
# name_normalized 중복 여부 확인
print("transfer_df name_normalized 중복 수:", transfer_df['name_normalized'].duplicated().sum())
print("fc25_df name_normalized 중복 수:", fc25_df['name_normalized'].duplicated().sum())

transfer_df name_normalized 중복 수: 1
fc25_df name_normalized 중복 수: 1


In [161]:
# 중복 제거
transfer_df = transfer_df.drop_duplicates(subset=['name_normalized'], keep='first')
fc25_df = fc25_df.drop_duplicates(subset=['name_normalized'], keep='first')

In [162]:
merged_transfer_fc_df = pd.merge(transfer_df, fc_df, on='name_normalized', suffixes=('_transfer', '_fc25'))
print(f"병합된 선수 수: {len(merged_transfer_fc_df)}")

병합된 선수 수: 3309


In [163]:
merged_transfer_fc_df

,name_normalized,name_transfer,age,field_transfer,position_transfer,nation_transfer,confederation,club_transfer,league,value_billion_krw,...,club_fc25,field_fc25,position_fc25,OVR,PAC,SHO,PAS,DRI,DEF,PHY
0,lamineyamal,Lamine Yamal,18,FW,Right_Winger,Spain,UEFA,FC Barcelona,LaLiga,3250.0,...,FC Barcelona,FW,Right_Winger,81,82,75,76,82,23,48
1,erlinghaaland,Erling Haaland,25,FW,Striker,Norway,UEFA,Manchester City,PremierLeague,2925.0,...,Manchester City,FW,Striker,91,88,92,70,81,45,88
2,kylianmbappe,Kylian Mbappe,26,FW,Striker,France,UEFA,Real Madrid,LaLiga,2925.0,...,Real Madrid,FW,Striker,91,97,90,80,92,36,78
3,bellinghamjude,Jude Bellingham,22,MF,Attacking_Midfielder,England,UEFA,Real Madrid,LaLiga,2925.0,...,Real Madrid,MF,Attacking_Midfielder,90,80,87,83,88,78,83
4,juniorvinicius,Vinicius Junior,25,FW,Left_Winger,Brazil,CONMEBOL,Real Madrid,LaLiga,2762.5,...,Real Madrid,FW,Left_Winger,90,95,84,81,91,29,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3304,boxallmichael,Michael Boxall,36,DF,Centre_Back,New Zealand,OFC,Minnesota United FC,MLS,2.4,...,Minnesota United,DF,Centre_Back,69,54,42,58,57,67,80
3305,niktzanev,Nik Tzanev,28,GK,Goalkeeper,New Zealand,OFC,Newport County,Other_leagues,2.0,...,Northampton,GK,Goalkeeper,60,61,59,56,61,46,59
3306,claytonlewis,Clayton Lewis,28,MF,Central_Midfielder,New Zealand,OFC,Without Club,Without_Club,1.6,...,Macarthur FC,MF,Defensive_Midfielder,68,70,58,69,68,61,76
3307,gabrielrodriguessloane,Gabriel Sloane-Rodrigues,18,FW,Right_Winger,New Zealand,OFC,Wellington Phoenix,A_League,1.6,...,Well. Phoenix,FW,Right_Winger,53,74,52,42,56,21,45


In [164]:
print(merged_transfer_fc_df.columns.tolist())

['name_normalized', 'name_transfer', 'age', 'field_transfer', 'position_transfer', 'nation_transfer', 'confederation', 'club_transfer', 'league', 'value_billion_krw', 'name_fc25', 'nation_fc25', 'club_fc25', 'field_fc25', 'position_fc25', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY']


In [166]:
new_column_order = ['name_transfer', 'age', 'nation_transfer', 'position_transfer', 'club_transfer', 'league', 'confederation', 'value_billion_krw', 'field_transfer', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'name_normalized', 'name_fc25', 'nation_fc25', 'club_fc25', 'field_fc25', 'position_fc25']
merged_transfer_fc_df = merged_transfer_fc_df[new_column_order]
merged_transfer_fc_df.head()

,name_transfer,age,nation_transfer,position_transfer,club_transfer,league,confederation,value_billion_krw,field_transfer,OVR,...,PAS,DRI,DEF,PHY,name_normalized,name_fc25,nation_fc25,club_fc25,field_fc25,position_fc25
0,Lamine Yamal,18,Spain,Right_Winger,FC Barcelona,LaLiga,UEFA,3250.0,FW,81,...,76,82,23,48,lamineyamal,Lamine Yamal,Spain,FC Barcelona,FW,Right_Winger
1,Erling Haaland,25,Norway,Striker,Manchester City,PremierLeague,UEFA,2925.0,FW,91,...,70,81,45,88,erlinghaaland,Erling Haaland,Norway,Manchester City,FW,Striker
2,Kylian Mbappe,26,France,Striker,Real Madrid,LaLiga,UEFA,2925.0,FW,91,...,80,92,36,78,kylianmbappe,Kylian Mbappe,France,Real Madrid,FW,Striker
3,Jude Bellingham,22,England,Attacking_Midfielder,Real Madrid,LaLiga,UEFA,2925.0,MF,90,...,83,88,78,83,bellinghamjude,Jude Bellingham,England,Real Madrid,MF,Attacking_Midfielder
4,Vinicius Junior,25,Brazil,Left_Winger,Real Madrid,LaLiga,CONMEBOL,2762.5,FW,90,...,81,91,29,69,juniorvinicius,Vinicius Junior,Brazil,Real Madrid,FW,Left_Winger


In [167]:
merged_transfer_fc_df = merged_transfer_fc_df.drop(columns=merged_transfer_fc_df.columns[-5:])
merged_transfer_fc_df.head()

,name_transfer,age,nation_transfer,position_transfer,club_transfer,league,confederation,value_billion_krw,field_transfer,OVR,PAC,SHO,PAS,DRI,DEF,PHY,name_normalized
0,Lamine Yamal,18,Spain,Right_Winger,FC Barcelona,LaLiga,UEFA,3250.0,FW,81,82,75,76,82,23,48,lamineyamal
1,Erling Haaland,25,Norway,Striker,Manchester City,PremierLeague,UEFA,2925.0,FW,91,88,92,70,81,45,88,erlinghaaland
2,Kylian Mbappe,26,France,Striker,Real Madrid,LaLiga,UEFA,2925.0,FW,91,97,90,80,92,36,78,kylianmbappe
3,Jude Bellingham,22,England,Attacking_Midfielder,Real Madrid,LaLiga,UEFA,2925.0,MF,90,80,87,83,88,78,83,bellinghamjude
4,Vinicius Junior,25,Brazil,Left_Winger,Real Madrid,LaLiga,CONMEBOL,2762.5,FW,90,95,84,81,91,29,69,juniorvinicius


In [168]:
merged_transfer_fc_df = merged_transfer_fc_df.rename(columns={
    'name_transfer': 'Name',
    'age': 'Age',
    'nation_transfer': 'Nation',
    'position_transfer': 'Position',
    'club_transfer': 'Club',
    'league': 'League',
    'confederation': 'Confederation',
    'value_billion_krw': 'Billion_KRW',
    'field_transfer': 'Field',
    'name_normalized': 'Name_ID'
})

In [171]:
merged_transfer_fc_df.head()

,Name,Age,Nation,Position,Club,League,Confederation,Billion_KRW,Field,OVR,PAC,SHO,PAS,DRI,DEF,PHY,Name_ID
0,Lamine Yamal,18,Spain,Right_Winger,FC Barcelona,LaLiga,UEFA,3250.0,FW,81,82,75,76,82,23,48,lamineyamal
1,Erling Haaland,25,Norway,Striker,Manchester City,PremierLeague,UEFA,2925.0,FW,91,88,92,70,81,45,88,erlinghaaland
2,Kylian Mbappe,26,France,Striker,Real Madrid,LaLiga,UEFA,2925.0,FW,91,97,90,80,92,36,78,kylianmbappe
3,Jude Bellingham,22,England,Attacking_Midfielder,Real Madrid,LaLiga,UEFA,2925.0,MF,90,80,87,83,88,78,83,bellinghamjude
4,Vinicius Junior,25,Brazil,Left_Winger,Real Madrid,LaLiga,CONMEBOL,2762.5,FW,90,95,84,81,91,29,69,juniorvinicius


In [172]:
merged_transfer_fc_df.to_csv('../data/football_player.csv', index=False, encoding='utf-8-sig')

In [126]:
# 모든 정보가 동일한 경우 필터링
allfit_rows = matched_pairs[(matched_pairs['nation_fc25'] == matched_pairs['nation_tfm']) &
                            (matched_pairs['field_fc25'] == matched_pairs['field_tfm']) &
                            (matched_pairs['position_fc25'] == matched_pairs['position_tfm']) &
                            (matched_pairs['club_fc25'] == matched_pairs['club_tfm'])]

allfit_rows.to_csv('../data/all_fits.csv', index=False, encoding='utf-8-sig')
allfit_rows

,name_fc25,field_fc25,position_fc25,nation_fc25,club_fc25,name_normalized,name_tfm,field_tfm,position_tfm,nation_tfm,club_tfm
0,Kylian Mbappe,FW,Striker,France,Real Madrid,kylianmbappe,Kylian Mbappe,FW,Striker,France,Real Madrid
1,Rodri,MF,Defensive_Midfielder,Spain,Manchester City,rodri,Rodri,MF,Defensive_Midfielder,Spain,Manchester City
2,Erling Haaland,FW,Striker,Norway,Manchester City,erlinghaaland,Erling Haaland,FW,Striker,Norway,Manchester City
3,Jude Bellingham,MF,Attacking_Midfielder,England,Real Madrid,bellinghamjude,Jude Bellingham,MF,Attacking_Midfielder,England,Real Madrid
4,Vinicius Junior,FW,Left_Winger,Brazil,Real Madrid,juniorvinicius,Vinicius Junior,FW,Left_Winger,Brazil,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...
3478,Jan Ziolkowski,DF,Centre_Back,Poland,Legia Warszawa,janziolkowski,Jan Ziolkowski,DF,Centre_Back,Poland,Legia Warszawa
3485,Boris Matic,DF,Right_Back,Serbia,Wolfsberger AC,borismatic,Boris Matic,DF,Right_Back,Serbia,Wolfsberger AC
3489,Kees Sims,GK,Goalkeeper,New Zealand,GAIS,keessims,Kees Sims,GK,Goalkeeper,New Zealand,GAIS
3497,Logan Farrington,FW,Striker,United States,FC Dallas,farringtonlogan,Logan Farrington,FW,Striker,United States,FC Dallas


In [90]:
# 1. 이름은 같지만 국적이 다른경우 필터링
mismatch_rows = matched_pairs[(matched_pairs['nation_fc25'] != matched_pairs['nation_tfm']) ]

#mismatch_rows.to_csv('../data/mismatched_nation.csv', index=False, encoding='utf-8-sig')
mismatch_rows

,name_fc25,field_fc25,position_fc25,nation_fc25,club_fc25,name_normalized,name_tfm,field_tfm,position_tfm,nation_tfm,club_tfm
46,Emiliano Martinez,GK,Goalkeeper,Argentina,Aston Villa,emilianomartinez,Emiliano Martinez,MF,Defensive_Midfielder,Uruguay,Sociedade Esportiva Palmeiras
65,Hakan Calhanoglu,MF,Defensive_Midfielder,Turkey,Lombardia FC,calhanogluhakan,Hakan Calhanoglu,MF,Defensive_Midfielder,Türkiye,Inter Milan
202,Otavio,MF,Attacking_Midfielder,Portugal,Al Nassr,otavio,Otavio,DF,Centre_Back,Brazil,Paris FC
251,Luis Suarez,FW,Striker,Uruguay,Inter Miami CF,luissuarez,Luis Suarez,FW,Striker,Colombia,UD Almería
283,Ferdi Kadioglu,DF,Left_Back,Turkey,Brighton,ferdikadioglu,Ferdi Kadioglu,DF,Left_Back,Türkiye,Brighton & Hove Albion
...,...,...,...,...,...,...,...,...,...,...,...
3496,Deniz Ertas,GK,Goalkeeper,Turkey,Konyaspor,denizertas,Deniz Ertas,GK,Goalkeeper,Türkiye,Konyaspor
3500,Amin Chiakha,FW,Striker,Denmark,F.C. København,aminchiakha,Amin Chiakha,FW,Striker,Algeria,FC Copenhagen
3507,Sydie Peck,MF,Central_Midfielder,Wales,Sheffield Utd,pecksydie,Sydie Peck,MF,Central_Midfielder,England,Sheffield United
3512,Erik Kojzek,FW,Striker,Slovenia,Wolfsberger AC,erikkojzek,Erik Kojzek,FW,Striker,Austria,Wolfsberger AC


In [91]:
# 2. 이름은 같지만 필드가 다른경우 필터링
mismatch_fild = matched_pairs[(matched_pairs['field_fc25'] != matched_pairs['field_tfm']) ]

#mismatch_rows.to_csv('../data/mismatched_fild.csv', index=False, encoding='utf-8-sig')
mismatch_fild

,name_fc25,field_fc25,position_fc25,nation_fc25,club_fc25,name_normalized,name_tfm,field_tfm,position_tfm,nation_tfm,club_tfm
10,Alisson,GK,Goalkeeper,Brazil,Liverpool,alisson,Alisson,FW,Right_Winger,Brazil,Shakhtar Donetsk
22,Ederson,GK,Goalkeeper,Brazil,Manchester City,ederson,Ederson,MF,Central_Midfielder,Brazil,Atalanta BC
33,Paulo Dybala,MF,Attacking_Midfielder,Argentina,AS Roma,dybalapaulo,Paulo Dybala,FW,Striker,Argentina,AS Roma
41,Marquinhos,DF,Centre_Back,Brazil,Paris SG,marquinhos,Marquinhos,FW,Left_Winger,Brazil,Spartak Moscow
42,Marquinhos,DF,Centre_Back,Brazil,Paris SG,marquinhos,Marquinhos,FW,Right_Winger,Brazil,Cruzeiro Esporte Clube
...,...,...,...,...,...,...,...,...,...,...,...
3475,Myles Lewis-Skelly,MF,Central_Midfielder,England,Arsenal,lewismylesskelly,Myles Lewis-Skelly,DF,Left_Back,England,Arsenal FC
3488,Paul Rothrock,MF,Side_Midfielder,United States,Sounders FC,paulrothrock,Paul Rothrock,FW,Left_Winger,United States,Seattle Sounders FC
3516,Kim Jae Min,MF,Central_Midfielder,"Korea, South",Jeju United,jaekimmin,Min-jae Kim,DF,Centre_Back,"Korea, South",Bayern Munich
3517,Wu Lei,MF,Defensive_Midfielder,China,Rongcheng FC,leiwu,Lei Wu,FW,Right_Winger,China,Shanghai Port


In [92]:
# 3. 이름은 같지만 포지션이 다른경우 필터링
mismatch_pos = matched_pairs[(matched_pairs['position_fc25'] != matched_pairs['position_tfm']) ]

#mismatch_rows.to_csv('../data/mismatched_pos.csv', index=False, encoding='utf-8-sig')
mismatch_pos

,name_fc25,field_fc25,position_fc25,nation_fc25,club_fc25,name_normalized,name_tfm,field_tfm,position_tfm,nation_tfm,club_tfm
5,Kevin De Bruyne,MF,Central_Midfielder,Belgium,Manchester City,bruynedekevin,Kevin De Bruyne,MF,Attacking_Midfielder,Belgium,SSC Napoli
7,Martin Odegaard,MF,Central_Midfielder,Norway,Arsenal,martinodegaard,Martin Odegaard,MF,Attacking_Midfielder,Norway,Arsenal FC
10,Alisson,GK,Goalkeeper,Brazil,Liverpool,alisson,Alisson,FW,Right_Winger,Brazil,Shakhtar Donetsk
22,Ederson,GK,Goalkeeper,Brazil,Manchester City,ederson,Ederson,MF,Central_Midfielder,Brazil,Atalanta BC
24,Bernardo Silva,MF,Central_Midfielder,Portugal,Manchester City,bernardosilva,Bernardo Silva,MF,Attacking_Midfielder,Portugal,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...
3503,Luis Toomey,MF,Attacking_Midfielder,New Zealand,Auckland FC,luistoomey,Luis Toomey,MF,Central_Midfielder,New Zealand,Without Club
3510,Emmanuel Agyemang,MF,Attacking_Midfielder,Ghana,Wolfsberger AC,agyemangemmanuel,Emmanuel Agyemang,MF,Central_Midfielder,Ghana,Wolfsberger AC
3516,Kim Jae Min,MF,Central_Midfielder,"Korea, South",Jeju United,jaekimmin,Min-jae Kim,DF,Centre_Back,"Korea, South",Bayern Munich
3517,Wu Lei,MF,Defensive_Midfielder,China,Rongcheng FC,leiwu,Lei Wu,FW,Right_Winger,China,Shanghai Port
